In [36]:
# from utils import *
import random
import Graph as G
import numpy as np

# Get vertices from DB and insert them into the graph
def Initialize_Random_Graph(DBPath='dataset/DB1K.csv',R=5):
    DBGraph=G.Graph()
    with open(DBPath, 'r') as f:
        DB = f.readlines()
        DB = [x.strip() for x in DB]
        for i,x in enumerate(DB):
            x = x.split(',')
            x= np.array(x,dtype=float)
            DBGraph.add_vertex(G.Vertex(int(x[0]),x[1:]))
            # Graph.Insert(x[0], x[1], x[2])
        # Choose R random neighbors for each vertex
        size= len(DBGraph.verticies)
        if(size==0 or size==1):
            return
        for vertex in DBGraph:
            for i in range(R):
                neighbor= DBGraph.get_vertex(int(random.random()*size))
                while(neighbor==vertex):
                    neighbor= DBGraph.get_vertex(int(random.random()*size))
                DBGraph.add_edge((vertex.key,vertex.value),(neighbor.key,neighbor.value),0)
    return DBGraph

#gets euclidean distance between 2 vectors
def get_distance(v1,v2):
    return  np.linalg.norm(v1-v2)

#gets medoid of a graph
def get_medoid(DBGraph):
    min_distance=10000000000000000000
    medoid=None
    for vertex in DBGraph:
        current_total_distance=0
        for vertex2 in DBGraph:
            if(vertex==vertex2):
                continue
            dist=get_distance(vertex.value,vertex2.value)
            current_total_distance+=dist
        
        if(current_total_distance<min_distance):
            min_distance=current_total_distance
            medoid=vertex
    return medoid,min_distance




#gets arg min distance from any vertex in Anyset to Query and min dist
def get_min_dist (Anyset,Query):
    min_dist=10000000000000000000
    min_vertex=None
    for vertex in Anyset:
        dist=get_distance(vertex.value,Query)
        if(dist<min_dist):
            min_dist=dist
            min_vertex=vertex
    return min_vertex,min_dist

#initially, start is the medoid
# s is a vertex, Query is a vector
# k is a number, L is a number

def Greedy_Search(start,Query,k,L):
    search_List={start}
    Visited=set()
    possible_frontier=search_List.difference(Visited)
    while possible_frontier != set():
        p_star,_= get_min_dist(possible_frontier,Query)
        #
        # print('pstar',p_star)
        if p_star==None:
            # break
            print('frontier: ')
            for v in possible_frontier:
                print(v)
            print(possible_frontier==set())
        search_List=search_List.union(p_star.neighbors)
        Visited.add(p_star)
        if(len(search_List)>L):
            #update search list to retain closes L points to x_q
            search_ListL_L=list(search_List)
            search_ListL_L.sort(key=lambda x: get_distance(x.value,Query))
            # only maintain L closest points
            search_ListL_L=search_ListL_L[:L]
            search_List=set(search_ListL_L)

        possible_frontier=search_List.difference(Visited)
        
    search_ListL_L=list(search_List)
    search_ListL_L.sort(key=lambda x: get_distance(x.value,Query))
    # only maintain k closest points
    search_ListL_L=search_ListL_L[:k]
    search_List=set(search_ListL_L)
    return search_List,Visited

def Robust_Prune(point,candidate_set,alpha,R):
    candidate_set=candidate_set.union(point.neighbors)
    candidate_set.remove(point)
    point.neighbors=set()
    while candidate_set!=set():
        p_star,min_dist= get_min_dist(candidate_set,point.value)
        point.neighbors.add(p_star)
        if(len(point.neighbors)==R):
            break
        DummySet=candidate_set.copy()
        for candidatePoint in candidate_set:
            if(alpha * get_distance(p_star.value,candidatePoint.value)<=get_distance(candidatePoint.value,point.value)):
                DummySet.remove(candidatePoint)
        candidate_set=DummySet


In [39]:
R=5
k=10
L=10

path='dataset/DB1K.csv'
DBGraph=Initialize_Random_Graph(path,R)

medoid,min_distance=get_medoid(DBGraph)
#how to choose query? how to choose k,L?

In [40]:
alpha=2
rng = np.random.default_rng(50)
print('dasatince')
for p in medoid.neighbors:
    print(get_distance(p.value,medoid.value))
Query=rng.random((1, 70), dtype=np.float32)
# def Greedy_Search(start,Query,k,L):
SL,Vis=Greedy_Search(medoid,Query,k,L)
print(medoid)
Robust_Prune(medoid,Vis,alpha,R)
print(medoid)
for p in medoid.neighbors:
    print(get_distance(p.value,medoid.value))

dasatince
3.0200763618325945
3.140710735566076
3.343679717556692
3.5195723555142324
2.904374072928623
994 neighbors: [883, 944, 649, 823, 131] 
 values: [0.23297 0.3855  0.26245]
min dist is  2.8492384808927453
neighbors are now {<Graph.Vertex object at 0x0000023456FEEF90>}
min dist is  2.904374072928623
neighbors are now {<Graph.Vertex object at 0x0000023456FEEF90>, <Graph.Vertex object at 0x0000023456FC9950>}
min dist is  2.9308057994176275
neighbors are now {<Graph.Vertex object at 0x0000023456FEEF90>, <Graph.Vertex object at 0x0000023456FC9950>, <Graph.Vertex object at 0x0000023456FFDA50>}
min dist is  2.9367920995024486
neighbors are now {<Graph.Vertex object at 0x0000023456FEEF90>, <Graph.Vertex object at 0x0000023447E59F50>, <Graph.Vertex object at 0x0000023456FC9950>, <Graph.Vertex object at 0x0000023456FFDA50>}
min dist is  3.0200763618325945
neighbors are now {<Graph.Vertex object at 0x0000023456FFDA50>, <Graph.Vertex object at 0x00000234570152D0>, <Graph.Vertex object at 0x0